***1. PRÉPARATION DES DONNÉES À LA SOURCE***

***1. Définition de la fonction nettoyage***

In [6]:
#!/usr/local/opt/python@3.9/bin/python3.9
#Inizializzazione
import pandas as pd


# from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import math
# import io


#1.1 Fonction de nettoyage des df

def nettoyage(df_to_clean, label):
    print("Nettoyage du jeu de données: \"%s\" " % label)
    llignes1, lcols1 = df_to_clean.shape
    print(llignes1, lcols1)
    df_to_clean_nac = df_to_clean.dropna(axis=1, how='all')
    llignes2, lcols2 = df_to_clean_nac.shape
    print("Après dropna, cols:", llignes2, lcols2)
    print("Colonnes de moins:\n", lcols1 - lcols2)

    df_to_clean_nacl = df_to_clean_nac.dropna(axis=0, how='all')
    llignes3, lcols3 = df_to_clean_nacl.shape
    print("Après dropna, lignes:", llignes3, lcols3)
    print("Lignes de moins:\n", llignes2 - llignes3)

    df_clean = df_to_clean_nacl.drop_duplicates()
    llignes4, lcols4 = df_clean.shape
    print("\nRemove duplicates -> Lignes de moins:%i" % (llignes2 - llignes4))
    df_clean.shape

    pd.set_option('display.float_format', '{:.2f}'.format)
    np.set_printoptions(precision=4)
    dcount_x = pd.DataFrame(df_clean.count()).reset_index().rename(columns={'index': 'Indicator', 0: 'count'})
    # print(dcount_x)
    dcount_3 = pd.Series(np.array(df_clean.count().tolist())*100/llignes3).rename('percentage')
    dcount_4 = pd.Series(llignes3-np.array(df_clean.count().tolist())).rename('vides')
    df_count = dcount_x.merge(dcount_3,left_index=True, right_index=True)
    df_count = df_count.merge(dcount_4, left_index=True, right_index=True)

    print(df_count)
    print("\n--------------------------------\n")

    return df_clean

def nettoyage_specifique(df_to_clean, list_f_to_clean):
    tmpdf = df_to_clean
    tmpdf.shape
    for i in list_f_to_clean:
        tmpdf = pd.DataFrame(tmpdf.loc[df_to_clean[i]>0])

    return tmpdf

***Importation des jeux de données***

In [7]:
# Importation des jeux de données
df_15_to_clean = pd.read_csv("data/2015-building-energy-benchmarking.csv", sep= ',', low_memory=False )
df_16_to_clean = pd.read_csv("data/2016-building-energy-benchmarking.csv", sep= ',', low_memory=False )

***Jeu des données 1: 2015***

In [8]:
# Traitement df 2015
# Je renomme certaines colonnes 
df_15_to_clean = df_15_to_clean.rename(columns={'GHGEmissions(MetricTonsCO2e)': 'TotalGHGEmissions', 'GHGEmissionsIntensity(kgCO2e/ft2)': 'GHGEmissionsIntensity', 'Seattle Police Department Micro Community Policing Plan Areas' : 'SPD MCPP Areas', 'SiteEUI(kBtu/sf)' : 'SiteEUI(kBtu_over_sf)', 'SiteEUIWN(kBtu/sf)':'SiteEUIWN(kBtu_over_sf)' })

# 1.2 Nettoyage et aperçu 
df_15_to_clean2 = nettoyage(df_15_to_clean, '2015')

Nettoyage du jeu de données: "2015" 
3340 47
Après dropna, cols: 3340 47
Colonnes de moins:
 0
Après dropna, lignes: 3340 47
Lignes de moins:
 0

Remove duplicates -> Lignes de moins:0
                          Indicator  count  percentage  vides
0                     OSEBuildingID   3340      100.00      0
1                          DataYear   3340      100.00      0
2                      BuildingType   3340      100.00      0
3               PrimaryPropertyType   3340      100.00      0
4                      PropertyName   3340      100.00      0
5     TaxParcelIdentificationNumber   3338       99.94      2
6                          Location   3340      100.00      0
7               CouncilDistrictCode   3340      100.00      0
8                      Neighborhood   3340      100.00      0
9                         YearBuilt   3340      100.00      0
10                NumberofBuildings   3340      100.00      0
11                   NumberofFloors   3332       99.76      8
12       

***Jeu des données 2: 2016***

In [15]:
# 1.2.2 Nettoyage et aperçu ( 2016 )
df_16_to_clean = df_16_to_clean.rename(columns={'SiteEUI(kBtu/sf)' : 'SiteEUI(kBtu_over_sf)', 'SiteEUIWN(kBtu/sf)':'SiteEUIWN(kBtu_over_sf)'})

# 1.2 Nettoyage et aperçu 
df_16_to_clean2 = nettoyage(df_16_to_clean, '2016')

Nettoyage du jeu de données: "2016" 
3376 46
Après dropna, cols: 3376 45
Colonnes de moins:
 1
Après dropna, lignes: 3376 45
Lignes de moins:
 0

Remove duplicates -> Lignes de moins:0
                          Indicator  count  percentage  vides
0                     OSEBuildingID   3376      100.00      0
1                          DataYear   3376      100.00      0
2                      BuildingType   3376      100.00      0
3               PrimaryPropertyType   3376      100.00      0
4                      PropertyName   3376      100.00      0
5                           Address   3376      100.00      0
6                              City   3376      100.00      0
7                             State   3376      100.00      0
8                           ZipCode   3360       99.53     16
9     TaxParcelIdentificationNumber   3376      100.00      0
10              CouncilDistrictCode   3376      100.00      0
11                     Neighborhood   3376      100.00      0
12       

***Définition des fonctions de traitement à appliquer sur les deux DF***

In [16]:
# Fonction de support qui aide à identifier des erreurs de format du df. 
# Notamment ceux qui peuvent emerger si le df est redefini par une fonction defectueuse

def controle_df(df):
    if isinstance(df, pd.DataFrame):
        print('ok')
    else:
        sys.exit("stop, check df")
        df.shape

In [17]:
def nonResidential(df): # 
    controle_df(df)
    # display(df.columns.values.tolist())
    display(df.shape)
    df.dropna(subset=['BuildingType'])
    print("dropna BuildingType")
    display(df.shape)
    
    res_dict = { 
  'SPS-District K-12': '1',
  'Nonresidential COS': '1',
  'Nonresidential WA': '1',
  'NonResidential': '1',
  'Multifamily MR (5-9)' : '0',
  'Multifamily LR (1-4)': '0', 
  'Multifamily HR (10+)': '0', 
  'Campus': '0'
  }

    df = pd.DataFrame(df)
    df['NonRes'] = df['BuildingType'].map(res_dict)
    df['NonRes'] = pd.to_numeric(df['NonRes'])
    
    display(df.groupby(['NonRes']).count())
    
    print("____________________")
    
    return pd.DataFrame(df)

def select_nonResidential(df):
    if isinstance(df, pd.DataFrame):
        print('ok')
        df = pd.DataFrame(df.loc[df['NonRes']==1])
        return pd.DataFrame(df)
        print("____________________")

    else:
        print('stop')

#On retient seulement les batiments non-res dans le DF
df_15_nr = select_nonResidential(nonResidential(df_15_to_clean2))
display(df_15_nr.shape)
df_16_nr = select_nonResidential(nonResidential(df_16_to_clean2))
display(df_16_nr.shape)


ok


(3340, 48)

dropna BuildingType


(3340, 48)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,...,GHGEmissionsIntensity,DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,SPD MCPP Areas,City Council Districts,SPD Beats,Zip Codes
NonRes,,,,,,,,,,,,,,,,,,,,,
0,1712,1712,1712,1712,1712,1711,1712,1712,1712,1712,...,1709,1712,1,1712,53,136,1711,132,1711,1712
1,1628,1628,1628,1628,1628,1627,1628,1628,1628,1628,...,1621,1627,12,1628,31,88,1627,81,1627,1628


...

dropna BuildingType


(3376, 45)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
NonRes,,,,,,,,,,,,,,,,,,,,,
0,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732,...,1725,1725,1725,1725,1725,1732,1732,15,1725,1725
1,1644,1644,1644,1644,1644,1644,1644,1644,1628,1644,...,1642,1642,1642,1642,1642,1644,1644,17,1642,1642


____________________
ok


(1644, 46)

***Manipulation des dataframes*** 

***CARACTÈRE RÉSIDENTIEL. On souhaite retenir seulement les bâtiments non-résidentiels. On associe la valeur 1 aux bâtiments non-résidentiels et 0 aux bâtiments résidentiels, puis on retient seulement les premiers.***

***CONFORMITÉ. Traitement de la donnée sur la conformité (ajout d'une colonne)***

In [18]:
def compliant(df):
    controle_df(df)
    
    compliant_dict = { 
  'Not-Compliant': '0',     
  'Non-Compliant': '0',
  'Missing Data': '0',
  'Error - Correct Default Data': '0',
  'Compliant': '1'}
  
    df['Compl']= df['ComplianceStatus'].map(compliant_dict)
    df['Compl'] = pd.to_numeric(df['Compl'])

    df_nr_c = pd.DataFrame(df.loc[(df['Compl']==1)])

    display(df_nr_c.groupby(['Compl']).count())

    # return pd.DataFrame(df)
    return pd.DataFrame(df_nr_c)

df_15_nr_compl = compliant(df_15_nr)
df_16_nr_compl = compliant(df_16_nr)


ok


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,...,DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,SPD MCPP Areas,City Council Districts,SPD Beats,Zip Codes,NonRes
Compl,,,,,,,,,,,,,,,,,,,,,
1,1628,1628,1628,1628,1628,1627,1628,1628,1628,1628,...,1627,12,1628,31,88,1627,81,1627,1628,1628


ok


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity,NonRes
Compl,,,,,,,,,,,,,,,,,,,,,
1,1524,1524,1524,1524,1524,1524,1524,1524,1511,1524,...,1524,1524,1524,1524,1524,1524,0,1524,1524,1524


***OUTLIERS. Correction des inhomogénéités dans la définition des outliers (ajout d'une colonne). <br>
ANCIENNETÉ. Calcul de l'ancienneté du bâtiment sur la base de l'année de construction (ajout d'une colonne). <br>
SURFACE EXPOSÉE. Estimation de la surface exposée en fonction de la surface couverte et du nombre d'étages (ajout d'une colonne). <br>
CONSTANCE DE CONSOMMATION. Selon l'usage de l'immeuble, j'ai classifié les immeubles en deux catégories : une à consommation constante et une à consommation différente entre jour et nuit. Finalement, ce volet n'a pas été inclus dans l'analyse que je présente.***

In [19]:
def manage_outl(df): 
    outliers_dict = { 
      'Low outlier': 'L',
      'Low Outlier': 'L',
      'High outlier': 'H',
      'High Outlier': 'H'}
    
    df['Outlier_ok'] = df['Outlier'].map(outliers_dict)
    
    return pd.DataFrame(df)

def anciennete(df, annee):
    df['Anciennete'] = int(annee) - df['YearBuilt']

    return pd.DataFrame(df)

def mesure_etages(df):
    df['no_zero_floors'] = df['NumberofFloors']
    df['no_zero_floors'] = df['no_zero_floors'].replace(0, 1)
    display(df['NumberofFloors'], df['no_zero_floors'])
    df['measure_floors'] = df['PropertyGFABuilding(s)'] / df['no_zero_floors'] + 10 * df['no_zero_floors'] * 4 * np.sqrt(df['PropertyGFABuilding(s)'] / df['no_zero_floors'])
                                                                                                                    
    return pd.DataFrame(df)

def constance_consommation(df):
    dict_consommation = {
        'Adult Education' : '1', 
        'Automobile Dealership' : '1', 
        'Bank Branch' : '1', 
        'College/University' : '1', 
        'Convention Center' : '1', 
        'Courthouse' : '1', 
        'Data Center' : '2', 
        'Distribution Center' : '2', 
        'Financial Office' : '1', 
        'Fire Station' : '2', 
        'Fitness Center/Health Club/Gym' : '1', 
        'Food Service' : '1', 
        'Hospital (General Medical & Surgical)' : '2', 
        'Hotel' : '2', 
        'K-12 School' : '1', 
        'Laboratory' : '1', 
        'Library' : '1', 
        'Lifestyle Center' : '1', 
        'Manufacturing/Industrial Plant' : '2', 
        'Medical Office' : '1', 
        'Movie Theater' : '1', 
        'Multifamily Housing' : '2', 
        'Museum' : '1', 
        'Non-Refrigerated Warehouse' : '1', 
        'Office' : '1', 
        'Other' : '1', 
        'Other - Education' : '1', 
        'Other - Entertainment/Public Assembly' : '1', 
        'Other - Lodging/Residential' : '2', 
        'Other - Mall' : '1', 
        'Other - Public Services' : '1', 
        'Other - Recreation' : '1', 
        'Other - Restaurant/Bar' : '1', 
        'Other - Services' : '1', 
        'Other - Utility' : '1', 
        'Other/Specialty Hospital' : '2', 
        'Outpatient Rehabilitation/Physical Therapy' : '1', 
        'Parking' : '1', 
        'Performing Arts' : '1', 
        'Personal Services (Health/Beauty, Dry Cleaning, etc)' : '1', 
        'Police Station' : '2', 
        'Pre-school/Daycare' : '1', 
        'Refrigerated Warehouse' : '2', 
        'Repair Services (Vehicle, Shoe, Locksmith, etc)' : '1', 
        'Residence Hall/Dormitory' : '2', 
        'Residential Care Facility' : '2', 
        'Restaurant' : '1', 
        'Retail Store' : '1', 
        'Self-Storage Facility' : '1', 
        'Senior Care Community' : '2', 
        'Social/Meeting Hall' : '1', 
        'Strip Mall' : '1', 
        'Supermarket/Grocery Store' : '1', 
        'Urgent Care/Clinic/Other Outpatient' : '2', 
        'Wholesale Club/Supercenter' : '1', 
        'Worship Facility' : '1'
    }
    
    df['Intensité de consommation'] = df['PrimaryPropertyType'].map(dict_consommation)
    
    return pd.DataFrame(df)


df_15_nr_fe = constance_consommation(mesure_etages(anciennete(manage_outl(df_15_nr_compl),2015)))
display(df_15_nr_fe.head(5))
display(df_15_nr_fe.shape)

df_16_nr_fe = constance_consommation(mesure_etages(anciennete(manage_outl(df_16_nr_compl), 2016)))
display(df_16_nr_fe.head(5))
display(df_16_nr_fe.shape)

0      12.00
1      11.00
2      41.00
3      10.00
4      18.00
        ... 
3313     NaN
3314    6.00
3319    4.00
3322    3.00
3334    2.00
Name: NumberofFloors, Length: 1628, dtype: float64

0      12.00
1      11.00
2      41.00
3      10.00
4      18.00
        ... 
3313     NaN
3314    6.00
3319    4.00
3322    3.00
3334    2.00
Name: no_zero_floors, Length: 1628, dtype: float64

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,...,City Council Districts,SPD Beats,Zip Codes,NonRes,Compl,Outlier_ok,Anciennete,no_zero_floors,measure_floors,Intensité de consommation
0,1,2015,NonResidential,Hotel,MAYFLOWER PARK HOTEL,659000030,"{'latitude': '47.61219025', 'longitude': '-122...",7,DOWNTOWN,1927,...,NaN,31.00,18081,1,1,NaN,88,12.00,48575.48,2
1,2,2015,NonResidential,Hotel,PARAMOUNT HOTEL,659000220,"{'latitude': '47.61310583', 'longitude': '-122...",7,DOWNTOWN,1996,...,NaN,31.00,18081,1,1,NaN,19,11.00,47512.52,2
2,3,2015,NonResidential,Hotel,WESTIN HOTEL,659000475,"{'latitude': '47.61334897', 'longitude': '-122...",7,DOWNTOWN,1969,...,NaN,31.00,18081,1,1,NaN,46,41.00,274673.33,2
3,5,2015,NonResidential,Hotel,HOTEL MAX,659000640,"{'latitude': '47.61421585', 'longitude': '-122...",7,DOWNTOWN,1926,...,NaN,31.00,18081,1,1,H,89,10.00,37454.84,2
4,8,2015,NonResidential,Hotel,WARWICK SEATTLE HOTEL,659000970,"{'latitude': '47.6137544', 'longitude': '-122....",7,DOWNTOWN,1980,...,NaN,31.00,19576,1,1,NaN,35,18.00,61591.93,2


(1628, 54)

0       12
1       11
2       41
3       10
4       18
        ..
3370     1
3372     1
3373     1
3374     1
3375     1
Name: NumberofFloors, Length: 1524, dtype: int64

0       12
1       11
2       41
3       10
4       18
        ..
3370     1
3372     1
3373     1
3374     1
3375     1
Name: no_zero_floors, Length: 1524, dtype: int64

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Outlier,TotalGHGEmissions,GHGEmissionsIntensity,NonRes,Compl,Outlier_ok,Anciennete,no_zero_floors,measure_floors,Intensité de consommation
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.00,0659000030,...,NaN,249.98,2.83,1,1,NaN,89,12,48575.48,2
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.00,0659000220,...,NaN,295.86,2.86,1,1,NaN,20,11,47512.52,2
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.00,0659000475,...,NaN,2089.28,2.19,1,1,NaN,47,41,241717.00,2
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.00,0659000640,...,NaN,286.43,4.67,1,1,NaN,90,10,37454.84,2
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.00,0659000970,...,NaN,505.01,2.88,1,1,NaN,36,18,63503.57,2


(1524, 52)

***Calcul des logarithmes***

In [21]:
list_to_log = ['PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)', 'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SiteEUI(kBtu_over_sf)', 'SiteEUIWN(kBtu_over_sf)', 'TotalGHGEmissions', 'GHGEmissionsIntensity', 'measure_floors', 'Anciennete']

def add_logs(df):
    for i in list_to_log:
        newname = 'log_%s' % i 
        df[newname] = np.log(df[i])
    return df



df_15_nr_fe_wlogs = add_logs(df_15_nr_fe)
df_16_nr_fe_wlogs = add_logs(df_16_nr_fe)

df_15_final = nettoyage_specifique(df_15_nr_fe_wlogs, list_to_log)
df_16_final = nettoyage_specifique(df_16_nr_fe_wlogs, list_to_log)

display(df_15_final.head(5))

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,...,log_PropertyGFAParking,log_PropertyGFABuilding(s),log_SiteEnergyUse(kBtu),log_SiteEnergyUseWN(kBtu),log_SiteEUI(kBtu_over_sf),log_SiteEUIWN(kBtu_over_sf),log_TotalGHGEmissions,log_GHGEmissionsIntensity,log_measure_floors,log_Anciennete
1,2,2015,NonResidential,Hotel,PARAMOUNT HOTEL,659000220,"{'latitude': '47.61310583', 'longitude': '-122...",7,DOWNTOWN,1996,...,9.62,11.39,15.94,15.99,4.55,4.60,5.57,0.87,10.77,2.94
4,8,2015,NonResidential,Hotel,WARWICK SEATTLE HOTEL,659000970,"{'latitude': '47.6137544', 'longitude': '-122....",7,DOWNTOWN,1980,...,9.43,11.58,16.51,16.53,4.79,4.80,6.23,1.39,11.03,3.56
5,9,2015,Nonresidential COS,Other,WEST PRECINCT (SEATTLE POLICE),660000560,"{'latitude': '47.6164389', 'longitude': '-122....",7,DOWNTOWN,1999,...,10.52,11.00,16.30,16.38,4.91,4.99,5.72,1.03,10.69,2.77
10,15,2015,NonResidential,Hotel,HOTEL MONACO,942000145,"{'latitude': '47.60712147', 'longitude': '-122...",7,DOWNTOWN,1969,...,9.87,11.80,16.50,16.56,4.62,4.67,6.36,1.28,11.01,3.83
11,16,2015,NonResidential,Hotel,W SEATTLE HOTEL,942000165,"{'latitude': '47.60718204', 'longitude': '-122...",7,DOWNTOWN,1998,...,11.02,12.51,17.14,17.15,4.41,4.42,6.58,0.66,11.65,2.83


***Exportations des DF separés***

In [22]:
df_15_final.to_csv("data/df_2015_clean.csv", sep=',')
df_16_final.to_csv("data/df_2016_clean.csv", sep=',')

***DF COMBINÉS***

In [23]:
# 1.3 Combinaison des df et nettoyage
# 1.3.1 : 2015+2016 concat

frames = [df_15_final, df_16_final]

# Noms des colonnes
# for i in frames:
#     print(i.columns.values.tolist())
#     print("____________")


df_15_et_16 = pd.concat(frames)
cols = ['OSEBuildingID', 'BuildingType', 'PrimaryPropertyType', 'PropertyName',
        'TaxParcelIdentificationNumber', 'Location', 'CouncilDistrictCode', 'Neighborhood',
        'YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking',
        'PropertyGFABuilding(s)', 'ListOfAllPropertyUseTypes', 'LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
        'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA',
        'YearsENERGYSTARCertified', 'ENERGYSTARScore', 'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SiteEUI(kBtu_over_sf)', 'SiteEUIWN(kBtu_over_sf)', 'SourceEUI(kBtu/sf)',
        'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)',
        'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)',
        'TotalGHGEmissions', 'GHGEmissionsIntensity',  
        'DefaultData',
        'Comment', 'ComplianceStatus', 'Outlier', '2010 Census Tracts',
        'SPD MCPP Areas', 'City Council Districts', 'SPD Beats', 'Zip Codes']

df_15_et_16_concat_to_clean = df_15_et_16.sort_values(by=cols)
df_15_et_16_concat = nettoyage(df_15_et_16_concat_to_clean, '2015+2016 concat')
df_15_et_16_concat.to_csv("data/df_2015_et_2016_concat_clean.csv", sep=',')


Nettoyage du jeu de données: "2015+2016 concat" 
660 71
Après dropna, cols: 660 71
Colonnes de moins:
 0
Après dropna, lignes: 660 71
Lignes de moins:
 0

Remove duplicates -> Lignes de moins:0
              Indicator  count  percentage  vides
0         OSEBuildingID    660      100.00      0
1              DataYear    660      100.00      0
2          BuildingType    660      100.00      0
3   PrimaryPropertyType    660      100.00      0
4          PropertyName    660      100.00      0
..                  ...    ...         ...    ...
66                 City    324       49.09    336
67                State    324       49.09    336
68              ZipCode    324       49.09    336
69             Latitude    324       49.09    336
70            Longitude    324       49.09    336

[71 rows x 4 columns]

--------------------------------



***Préparation du jeu de données pour l'étude de l'Energy Star Score***

In [29]:
df_15_et_16_concat_star = df_15_et_16_concat.dropna(subset=['ENERGYSTARScore'])
display(df_15_et_16_concat_star.shape)
display(df_15_et_16_concat.shape)
df_15_et_16_concat_star.to_csv("data/df_2015_et_2016_concat_star_clean.csv", sep=',')

(498, 71)

(660, 71)

In [31]:
display(df_15_et_16_concat_star['ENERGYSTARScore'])

1      51.00
1      61.00
2      43.00
4      67.00
4      75.00
        ... 
3320   99.00
3324   98.00
3329   98.00
3337   72.00
3338   37.00
Name: ENERGYSTARScore, Length: 498, dtype: float64

***Dans le jeu suivant, on combine côte à côte les jeux des deux années. Ce jeu n'a pas été utilisé.***

In [25]:
# 1.3.2 : 2015+2016 merged
df_m_to_clean = pd.merge(df_15_final, df_16_final, how='outer', left_index=True, right_index=True, suffixes=('_2015', '_2016'))
df_15_et_16_merged = nettoyage(df_m_to_clean, '2015+2016 merged')
display(df_15_et_16_merged.head(5))
df_15_et_16_merged.to_csv("data/df_2015_et_2016_merged_clean.csv", sep=',')

Nettoyage du jeu de données: "2015+2016 merged" 
530 128
Après dropna, cols: 530 126
Colonnes de moins:
 2
Après dropna, lignes: 530 126
Lignes de moins:
 0

Remove duplicates -> Lignes de moins:0
                            Indicator  count  percentage  vides
0                  OSEBuildingID_2015    336       63.40    194
1                       DataYear_2015    336       63.40    194
2                   BuildingType_2015    336       63.40    194
3            PrimaryPropertyType_2015    336       63.40    194
4                   PropertyName_2015    336       63.40    194
..                                ...    ...         ...    ...
121  log_SiteEUIWN(kBtu_over_sf)_2016    324       61.13    206
122        log_TotalGHGEmissions_2016    324       61.13    206
123    log_GHGEmissionsIntensity_2016    324       61.13    206
124           log_measure_floors_2016    324       61.13    206
125               log_Anciennete_2016    324       61.13    206

[126 rows x 4 columns]

----------

,OSEBuildingID_2015,DataYear_2015,BuildingType_2015,PrimaryPropertyType_2015,PropertyName_2015,TaxParcelIdentificationNumber_2015,Location,CouncilDistrictCode_2015,Neighborhood_2015,YearBuilt_2015,...,log_PropertyGFAParking_2016,log_PropertyGFABuilding(s)_2016,log_SiteEnergyUse(kBtu)_2016,log_SiteEnergyUseWN(kBtu)_2016,log_SiteEUI(kBtu_over_sf)_2016,log_SiteEUIWN(kBtu_over_sf)_2016,log_TotalGHGEmissions_2016,log_GHGEmissionsIntensity_2016,log_measure_floors_2016,log_Anciennete_2016
1,2.00,2015.00,NonResidential,Hotel,PARAMOUNT HOTEL,659000220,"{'latitude': '47.61310583', 'longitude': '-122...",7.00,DOWNTOWN,1996.00,...,9.62,11.39,15.94,15.97,4.55,4.58,5.69,1.05,10.77,3.00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.19,13.54,18.10,18.12,4.56,4.58,7.64,0.78,12.40,3.85
4,8.00,2015.00,NonResidential,Hotel,WARWICK SEATTLE HOTEL,659000970,"{'latitude': '47.6137544', 'longitude': '-122....",7.00,DOWNTOWN,1980.00,...,11.03,11.64,16.47,16.50,4.74,4.78,6.22,1.06,11.06,3.58
5,9.00,2015.00,Nonresidential COS,Other,WEST PRECINCT (SEATTLE POLICE),660000560,"{'latitude': '47.6164389', 'longitude': '-122....",7.00,DOWNTOWN,1999.00,...,10.52,11.00,16.31,16.35,4.91,4.95,5.71,1.13,10.69,2.83
10,15.00,2015.00,NonResidential,Hotel,HOTEL MONACO,942000145,"{'latitude': '47.60712147', 'longitude': '-122...",7.00,DOWNTOWN,1969.00,...,9.87,11.80,16.59,16.63,4.78,4.82,6.54,1.51,11.01,3.85
